In [39]:
import nflreadpy as nfl
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [40]:
# load play by play data, convert to pandas
pbp = nfl.load_pbp(seasons=[2025])
df = pbp.to_pandas()

# player stats
ps = nfl.load_player_stats([2025]).to_pandas()
#print(ps.head(1))

# team stats
ts = nfl.load_team_stats([2025]).to_pandas()
#print(ts.head(1))

# import roster
roster_df = nfl.load_rosters(2025).to_pandas()

# Seahawks offensive data
seahawks = df[df.posteam == 'SEA']

In [41]:
# features we have available to us
for col in seahawks.columns:
    print(col)

play_id
game_id
old_game_id
home_team
away_team
season_type
week
posteam
posteam_type
defteam
side_of_field
yardline_100
game_date
quarter_seconds_remaining
half_seconds_remaining
game_seconds_remaining
game_half
quarter_end
drive
sp
qtr
down
goal_to_go
time
yrdln
ydstogo
ydsnet
desc
play_type
yards_gained
shotgun
no_huddle
qb_dropback
qb_kneel
qb_spike
qb_scramble
pass_length
pass_location
air_yards
yards_after_catch
run_location
run_gap
field_goal_result
kick_distance
extra_point_result
two_point_conv_result
home_timeouts_remaining
away_timeouts_remaining
timeout
timeout_team
td_team
td_player_name
td_player_id
posteam_timeouts_remaining
defteam_timeouts_remaining
total_home_score
total_away_score
posteam_score
defteam_score
score_differential
posteam_score_post
defteam_score_post
score_differential_post
no_score_prob
opp_fg_prob
opp_safety_prob
opp_td_prob
fg_prob
safety_prob
td_prob
extra_point_prob
two_point_conversion_prob
ep
epa
total_home_epa
total_away_epa
total_home_rush_epa


In [42]:
# define var for all pass & run plays (exclude special teams, etc)
sea_off = seahawks[seahawks.play_type.isin(["pass", "run"])].copy()

# dedicated pass/run feature
sea_off['run_pass'] = sea_off.apply(
    lambda x: 'run' if x['rush_attempt'] == 1 else ('pass' if x['pass_attempt'] == 1 else 'other'),
    axis=1
)

print("--- Play Types Breakdown ---\n" + f"\n {sea_off['play_type'].value_counts()}")

# run/pass breakdown on each down
summary = (sea_off
           .groupby(['down', 'play_type'])
           .size()
           .unstack()
           .reset_index()
)

summary['total'] = summary['run'] + summary['pass']
summary['pct_pass'] = (summary['pass'] / summary['total']) * 100
summary['pct_run'] = (summary['run'] / summary['total']) * 100

print(f"\n\n{pd.DataFrame(summary)}")

tableau_cols = [
    'game_id', 'week', 'season_type', 'game_date',
    'posteam', 'defteam', 'home_team', 'away_team',
    'qtr', 'down', 'ydstogo', 'yardline_100',
    'quarter_seconds_remaining', 'game_seconds_remaining', 'score_differential',
    'play_id', 'play_type', 'desc',
    'shotgun', 'no_huddle', 'qb_dropback', 'qb_scramble',
    'pass_attempt', 'complete_pass', 'incomplete_pass',
    'passer_player_name', 'receiver_player_name', 'receiver_player_id',
    'passing_yards', 'air_yards', 'yards_after_catch',
    'pass_location', 'pass_length',
    'interception', 'sack', 'touchdown', 'pass_touchdown',
    'rush_attempt', 'rushing_yards', 'rush_touchdown',
    'run_location', 'run_gap', 'rusher_player_name', 'rusher_player_id',
    'epa', 'wpa', 'success', 'cpoe', 'first_down',
    'xyac_epa', 'xyac_mean_yardage',
    'fumble', 'fumble_lost', 'penalty'
]

sea_tableau = sea_off[tableau_cols].copy()
sea_tableau['is_play_action'] = (
    (sea_tableau['shotgun'] == 0) & 
    (sea_tableau['pass_attempt'] == 1) & 
    (sea_tableau['down'] <= 2)
).astype(int)

sea_tableau.to_csv('data/seahawksoffense.csv')

--- Play Types Breakdown ---

 pass    374
run     373
Name: play_type, dtype: int64


play_type  down  pass  run  total   pct_pass    pct_run
0           1.0   152  197    349  43.553009  56.446991
1           2.0   113  125    238  47.478992  52.521008
2           3.0   104   47    151  68.874172  31.125828
3           4.0     5    4      9  55.555556  44.444444


In [43]:
wrs_summary = sea_off.groupby(['receiver_player_name', 'yards_gained']).size().unstack(fill_value=0).reset_index()

print(wrs_summary)

yards_gained receiver_player_name  0.0  2.0  3.0  4.0  5.0  6.0  7.0  8.0  \
0                        A.Barner   11    2    1    3    4    4    5    1   
1                          C.Kupp   17    0    2    0    2    4    4    1   
2                         C.White    1    0    0    0    0    0    0    0   
3                         D.Young    0    0    0    0    0    0    0    0   
4                        E.Arroyo   11    0    1    0    2    2    2    1   
5                       E.Saubert    0    0    0    0    0    0    0    0   
6                        G.Holani    0    0    0    1    0    0    0    0   
7                          J.Bobo    0    0    1    0    0    0    0    0   
8                  J.Smith-Njigba   32    4    0    7    1    2    3    6   
9                        K.Walker    7    1    3    1    3    0    0    0   
10                     N.Kallerup    1    0    0    0    0    0    0    0   
11                      R.Shaheed    8    0    1    0    0    2    1    0   

In [44]:
# group ydstogo into new "distance_bucket"
sea_off['distance_bucket'] = pd.cut(sea_off['ydstogo'],
                                    bins=[0,1,3,6,10,100],
                                    labels=['0-1', '2-3', '4-6', '7-10', '11+'])
# 1, Short, Medium, Long, Extra Long


dnd_summary = (sea_off
.groupby(['down', 'distance_bucket', 'run_pass'])
.size()
.unstack(fill_value=0)
.reset_index())

# down and distance run/pass breakdowns
dnd_summary['total'] = dnd_summary['run'] + dnd_summary['pass']
dnd_summary['pct_pass'] = (dnd_summary['pass'] / dnd_summary['total']) * 100
dnd_summary['pct_run'] = (dnd_summary['run'] / dnd_summary['total']) * 100

dnd_summary.dropna(axis=0, inplace=True)
print("--- D&D Pass/Run Breakdown ---")
print(dnd_summary)

--- D&D Pass/Run Breakdown ---
run_pass  down distance_bucket  pass  run  total    pct_pass     pct_run
0          1.0             0-1     0    5      5    0.000000  100.000000
1          1.0             2-3     3    3      6   50.000000   50.000000
2          1.0             4-6     4    6     10   40.000000   60.000000
3          1.0            7-10   137  177    314   43.630573   56.369427
4          1.0             11+     8    6     14   57.142857   42.857143
5          2.0             0-1     5    9     14   35.714286   64.285714
6          2.0             2-3     6   20     26   23.076923   76.923077
7          2.0             4-6    18   36     54   33.333333   66.666667
8          2.0            7-10    57   49    106   53.773585   46.226415
9          2.0             11+    27   11     38   71.052632   28.947368
10         3.0             0-1     6   15     21   28.571429   71.428571
11         3.0             2-3    15   11     26   57.692308   42.307692
12         3.0      

In [45]:
# play action analyses
print(f"Loaded {len(sea_off)} total Seahawks offensive plays (pass/run) through Week 7.\n")

# 1) Play-Action Analysis w/ a proxy (under-center, 1st | 2nd down)
print("--- 1. Play-Action Analysis (w/ proxy)---")
print('Proxy: under-center, 1st or 2nd down\n')

# can alter as season goes on
qb = 'S.Darnold'

# grab the passing plays w/ darnold
sea_pass = sea_off[(sea_off['passer_player_name'] == qb) & (sea_off['play_type'] == 'pass')].copy()

# proxy => under-center, 1st | 2nd down
if not sea_pass.empty:
    is_pa_proxy = (sea_pass['shotgun'] == 0) & (sea_pass['down'] <= 2)
    # define pa plays & non pa plays
    pa_passes = sea_pass[is_pa_proxy]
    non_pa_passes = sea_pass[~is_pa_proxy]

    pa_frequency = len(pa_passes) / len(sea_pass)
    print(f"'Play-Action' Frequency: {pa_frequency:.2%}")

    # calcuate epa for pa & non pa
    pa_epa = pa_passes['epa'].mean()
    non_pa_epa = non_pa_passes['epa'].mean()
    print(f"w/ play-action: {pa_epa:.2f} EPA/Play on {len(pa_passes)} plays.")
    print(f"w/o play-action: {non_pa_epa:.2f} EPA/Play on {len(non_pa_passes)} plays.")
else:
    print(f"No pass plays found for fraud {qb}.")

# PA on downs
# breakdown of play action by down
if not pa_passes.empty:
    pa_down_breakdown = pa_passes.groupby('down').agg(
        pa_plays=('play_id', 'count')
    )

    total_by_down = sea_pass.groupby('down').agg(
        total_plays=('play_id', 'count')
    )

    pa_down_summary = pa_down_breakdown.join(total_by_down, how='outer').fillna(0)
    pa_down_summary['pa_rate_%'] = (pa_down_summary['pa_plays'] / pa_down_summary['total_plays']) * 100

    print("\nPlay-Action Breakdown by Down:")
    print(pa_down_summary.round(2))
else:
    print("\nNo play-action proxy plays found.")

Loaded 747 total Seahawks offensive plays (pass/run) through Week 7.

--- 1. Play-Action Analysis (w/ proxy)---
Proxy: under-center, 1st or 2nd down

'Play-Action' Frequency: 30.81%
w/ play-action: 0.43 EPA/Play on 114 plays.
w/o play-action: 0.08 EPA/Play on 256 plays.

Play-Action Breakdown by Down:
      pa_plays  total_plays  pa_rate_%
down                                  
1.0       77.0          150      51.33
2.0       37.0          112      33.04
3.0        0.0          103       0.00
4.0        0.0            5       0.00


In [46]:
# 2) Target Share Analysis for JSN
print("--- 2. Target Share Analysis ---")
sea_receivers = sea_off[sea_off['receiver_player_name'].notna()].copy()

total_targets = len(sea_receivers)
player_targets = sea_receivers.groupby('receiver_player_name').size().sort_values(ascending=False)
player_target_share = (player_targets / total_targets) * 100

print("Overall Target Share (Weeks 1-6):")
print(player_target_share.head(5).to_string(float_format="%.2f%%"))
print("-" * 20)

# situational target share (3rd/4th, 4+ yds to go)
situational_targets = sea_receivers[
    ((sea_receivers['down'] == 3)| (sea_receivers['down'] == 4)) & (sea_receivers['ydstogo'] >= 4)
]

total_situational_targets = len(situational_targets)
situational_player_targets = situational_targets.groupby('receiver_player_name').size()
situational_player_share = (situational_player_targets / total_situational_targets) * 100

print("\nSituational Target Share (3rd/4th & 4+ yards):")
print(situational_player_share.sort_values(ascending=False).head(5).to_string(float_format="%.2f%%"))


--- 2. Target Share Analysis ---
Overall Target Share (Weeks 1-6):
receiver_player_name
J.Smith-Njigba   35.69%
C.Kupp           15.63%
A.Barner         14.75%
E.Arroyo          7.67%
K.Walker          7.67%
--------------------

Situational Target Share (3rd/4th & 4+ yards):
receiver_player_name
J.Smith-Njigba   36.99%
C.Kupp           17.81%
A.Barner         12.33%
T.Horton          8.22%
K.Walker          5.48%


In [47]:
# 3) Target Share (Play-Action)
print("--- 3. Play-Action Target Share ---")

if not pa_passes.empty:
    # only keep plays with a valid target
    pa_targets = pa_passes[pa_passes['receiver_player_name'].notna()].copy()

    total_pa_targets = len(pa_targets)
    receiver_pa_counts = pa_targets.groupby('receiver_player_name').size().sort_values(ascending=False)
    receiver_pa_share = (receiver_pa_counts / total_pa_targets) * 100

    print("Target Share on Play-Action Passes:")
    print(receiver_pa_share.head(5).to_string(float_format="%.2f%%"))
else:
    print("No play-action proxy plays found for target share.")

--- 3. Play-Action Target Share ---
Target Share on Play-Action Passes:
receiver_player_name
J.Smith-Njigba   35.00%
E.Arroyo         16.00%
A.Barner         15.00%
C.Kupp            9.00%
K.Walker          9.00%


In [48]:
# JSN pass play analysis, where on the field, how far
# Where is JSN on 3rd/4th & 4+ yards?
# 'pass_location' and 'air_yards'

print("--- 3. JSN Analysis ---")
jsn_plays = sea_off[sea_off['receiver_player_name'] == 'J.Smith-Njigba'].copy()

# general JSN stats
print(f'Percentage of Passing Plays to JSN: {round((len(jsn_plays)) / (len(sea_off[sea_off["play_type"] == "pass"])), 2) * 100}%')
jsn_catch_pct = ((jsn_plays['incomplete_pass'] == 0).sum() / len(jsn_plays)) * 100
print(f"JSN Catch %: {jsn_catch_pct:.2f}%")
print(f"Explosive Play % (15+ yds): {(jsn_plays['yards_gained'] > 15).mean() * 100:.2f}%\n")

print("-" * 20)


# where on the field and how far?
jsn_rec_analysis = jsn_plays.groupby('pass_location').agg(
    pass_location=('pass_location', 'count'),
    pass_dist=('air_yards', 'mean'),
    catch_pct=('incomplete_pass', lambda x: (x == 0).mean() * 100)
)
print()
print(jsn_rec_analysis)

# by down, how far?
jsn_by_down = jsn_plays.groupby('down').agg(
    targets=('down', 'count'),
    catch_pct=('incomplete_pass', lambda x: (x == 0).mean() * 100),
    avg_air_yards=('air_yards', 'mean'),
    avg_yards_gained=('yards_gained', 'mean')
).round(2)

print()
print(jsn_by_down)


print("\n" + "-" * 20)

print(f"\nAverage Expected Points Added (EPA): {jsn_plays['epa'].mean():.2f}")

print("\n" + "-" * 20)

jsn_by_formation = jsn_plays.groupby('shotgun').agg(
    catches=('complete_pass', 'sum'),
    avg_air_yards=('air_yards', 'mean'),
    avg_epa=('epa', 'mean')
).round(2)

print('\nEPA by formation')
print(jsn_by_formation)

print("\n" + "-" * 20)

jsn_shot = jsn_plays[jsn_plays['shotgun'] == 1].copy()
jsn_uc = jsn_plays[jsn_plays['shotgun'] == 0].copy()

print('\nShotgun Analysis')
# where on the field (shotgun)
jsn_shot_analysis = jsn_shot.groupby('pass_location').agg(
    pass_location=('pass_location', 'count'),
    pass_dist=('air_yards', 'mean'),
    catch_pct=('incomplete_pass', lambda x: (x == 0).mean() * 100)
)
print(jsn_shot_analysis)

print("\n" + "-" * 20)

print('\nUnder Center Analysis')
# where on the field (uc)
jsn_uc_analysis = jsn_plays.groupby('pass_location').agg(
    pass_count=('pass_location', 'count'),
    pass_dist=('air_yards', 'mean'),
    catch_pct=('incomplete_pass', lambda x: (x == 0).mean() * 100)
)
print(jsn_uc_analysis)


--- 3. JSN Analysis ---
Percentage of Passing Plays to JSN: 32.0%
JSN Catch %: 76.03%
Explosive Play % (15+ yds): 26.45%

--------------------

               pass_location  pass_dist  catch_pct
pass_location                                     
left                      60  11.633333  71.666667
middle                    21  15.428571  90.476190
right                     40  10.925000  75.000000

      targets  catch_pct  avg_air_yards  avg_yards_gained
down                                                     
1.0        49      81.63          12.80             12.71
2.0        41      75.61          12.12             12.17
3.0        30      66.67          10.60              9.60
4.0         1     100.00          17.00             18.00

--------------------

Average Expected Points Added (EPA): 0.63

--------------------

EPA by formation
         catches  avg_air_yards  avg_epa
shotgun                                 
0.0         27.0          16.59     0.90
1.0         62.0        

In [49]:
# running game analysis, where do they run
#  use 'run_location', 'run_gap', 'rushing_yards, 'ydstogo'

print("--- 4. Run Game Analysis ---")
sea_runs = sea_off[(sea_off['rusher_player_name'] == 'K.Walker') | (sea_off['rusher_player_name'] == 'Z.Charbonnet')].copy()
walker_runs = sea_runs[sea_runs['rusher_player_name'] == 'K.Walker'].copy()
charb_runs = sea_runs[sea_runs['rusher_player_name'] == 'Z.Charbonnet'].copy()

first_and_not_ten = sea_runs[(sea_runs['down'] == 1) & (sea_runs['ydstogo'] != 10)].index
#sea_runs.drop(first_and_not_ten, inplace=True)

# define a successful run
    # a run is succesCardsul if it completed the following:
    # 1st down gain >= 40% of ydstogo, 2nd down gain >= 60% ydstogo, 3rd/4th converted a first down
sea_runs['successful'] = (
    ((sea_runs['down'] == 1) & (sea_runs['yards_gained'] >= 0.4 * sea_runs['ydstogo'])) |
    ((sea_runs['down'] == 2) & (sea_runs['yards_gained'] >= 0.6 * sea_runs['ydstogo'])) |
    ((sea_runs['down'].isin([3,4])) & (sea_runs['yards_gained'] >= sea_runs['ydstogo']))
)

# general run info
print(f"Rushes: {len(sea_runs)}")
print(f"Yards Per Carry: {sea_runs['yards_gained'].mean():.2f}")
print(f"Explosive Run % (>=10 yds): {(sea_runs['yards_gained'] >= 10).mean() * 100:.2f}%")

# runs by down
print("\n Runs by Down")
run_by_down = sea_runs.groupby('down').agg(
    attempts=('play_type', 'count'),
    ypc=('yards_gained', 'mean'),
    success_rate=('successful', lambda x: x.mean() * 100)
).round(2)

run_by_down['run_rate_%'] = (run_by_down['attempts'] / len(sea_runs)) * 100
print(run_by_down)

print("\n" + "-" * 20)

# individual RBs analysis
print('=== K.Walker Analysis ===')
print(f"K.Walker Rushes: {len(walker_runs)}")
print(f"Yards Per Carry: {walker_runs['yards_gained'].mean():.2f}")
print(f"Explosive Run % (>=10 yds): {(walker_runs['yards_gained'] >= 10).mean() * 100:.2f}%")
print("\n" + "-" * 20)

print('=== Z.Charbonnet Analysis ===')
print(f"Z.Charbonnet Rushes: {len(charb_runs)}")
print(f"Yards Per Carry: {charb_runs['yards_gained'].mean():.2f}")
print(f"Explosive Run % (>=10 yds): {(charb_runs['yards_gained'] >= 10).mean() * 100:.2f}%")

print()

--- 4. Run Game Analysis ---
Rushes: 302
Yards Per Carry: 4.12
Explosive Run % (>=10 yds): 10.60%

 Runs by Down
      attempts   ypc  success_rate  run_rate_%
down                                          
1.0        167  4.43         47.90   55.298013
2.0        106  3.79         45.28   35.099338
3.0         28  3.71         35.71    9.271523
4.0          1  0.00          0.00    0.331126

--------------------
=== K.Walker Analysis ===
K.Walker Rushes: 170
Yards Per Carry: 4.48
Explosive Run % (>=10 yds): 14.71%

--------------------
=== Z.Charbonnet Analysis ===
Z.Charbonnet Rushes: 132
Yards Per Carry: 3.66
Explosive Run % (>=10 yds): 5.30%



In [50]:
# redzone ball distribution analysis inside the 20

In [51]:
# no hudlle play and ball distribution analysis

In [52]:
# sam darnold scramble completion % vs in pocket
# qb

In [53]:
# qtr play breakdown analysis

In [54]:
# what plays they run on 3rd

In [55]:
jsn_catches = sea_pass[
    (sea_pass['receiver_player_name'] == 'J.Smith-Njigba') &
    (sea_pass['complete_pass'] == 1)
]

# 
formation_breakdown = (
    jsn_catches.groupby('shotgun')
    .agg(
        total_catches=('play_id', 'count'),
        avg_air_yards=('air_yards', 'mean'),
        avg_yards_after_catch=('yards_after_catch', 'mean'),
        avg_epa=('epa', 'mean')
    )
    .reset_index()
)

formation_breakdown['formation'] = formation_breakdown['shotgun'].map({0: 'Under Center', 1: 'Shotgun'})

formation_breakdown = formation_breakdown[['formation', 'total_catches', 'avg_air_yards', 'avg_yards_after_catch', 'avg_epa']]
print("JSN's Formation Breakdown")
print(formation_breakdown)

JSN's Formation Breakdown
      formation  total_catches  avg_air_yards  avg_yards_after_catch   avg_epa
0  Under Center             27      16.925926               5.629630  1.832767
1       Shotgun             62       9.387097               3.822581  1.004096


In [56]:
print("--- 4. 3rd Down Target Breakdown ---")

# Filter for all 3rd down pass plays
third_down_passes = sea_off[sea_off['down'] == 3].copy()

if third_down_passes.empty:
    print("No 3rd down passing plays found.")
else:
    # drop plays where the receiver isn't listed
    third_down_targets = third_down_passes.dropna(subset=['receiver_player_name'])

    if third_down_targets.empty:
        print("No 3rd down targets found.")
    else:
        # group by receiver and  key stats
        third_down_analysis = third_down_targets.groupby('receiver_player_name').agg(
            targets=('play_id', 'count'),
            receptions=('complete_pass', 'sum'),
            catch_pct=('incomplete_pass', lambda x: (x == 0).mean() * 100),
            total_yards=('yards_gained', 'sum'),
            avg_air_yards=('air_yards', 'mean'),
            first_downs=('first_down', 'sum'),
            avg_epa=('epa', 'mean')
        ).round(2)

        # sort by targets to see the top toptions
        third_down_analysis_sorted = third_down_analysis.sort_values(by='targets', ascending=False)

        print("Target Distribution on ALL 3rd Downs:")
        print(third_down_analysis_sorted)
        print("-" * 20)

jsn_plays['third_down_converted'].sum()



--- 4. 3rd Down Target Breakdown ---
Target Distribution on ALL 3rd Downs:
                      targets  receptions  catch_pct  total_yards  \
receiver_player_name                                                
J.Smith-Njigba             30        19.0      66.67        288.0   
C.Kupp                     19        10.0      52.63        146.0   
A.Barner                   12        12.0     100.00         99.0   
E.Arroyo                    7         3.0      71.43         21.0   
T.Horton                    6         4.0      66.67         55.0   
K.Walker                    6         5.0      83.33         46.0   
R.Shaheed                   5         2.0      40.00         39.0   
Z.Charbonnet                4         4.0     100.00         23.0   
C.White                     2         1.0     100.00         21.0   
G.Holani                    1         1.0     100.00          4.0   

                      avg_air_yards  first_downs  avg_epa  
receiver_player_name                

15.0

In [57]:
print("--- 5. JSN Down & Distance Analysis ---")

jsn_down_dist = jsn_plays[
    (jsn_plays['down'] > 0) & 
    (jsn_plays['play_type'] == 'pass')
].copy()

if not jsn_down_dist.empty:
    # Group by both 'down' and 'ydstogo' (yards to go for a 1st)
    jsn_dd_analysis = jsn_down_dist.groupby(['down', 'distance_bucket']).agg(
    targets=pd.NamedAgg(column='play_id', aggfunc='count'),
    catch_pct=pd.NamedAgg(column='incomplete_pass', aggfunc=lambda x: (x == 0).mean() * 100),
    avg_air_yards=pd.NamedAgg(column='air_yards', aggfunc='mean')
    ).sort_values(by=['down', 'distance_bucket'])
    
    print("\nJSN Targets by Down and Distance:")
    
    # Use pd.option_context to display all rows
    with pd.option_context('display.max_rows', None):
        print(jsn_dd_analysis.round(2))

else:
    print("No J.Smith-Njigba plays met the requirements.")

print("-" * 20)

--- 5. JSN Down & Distance Analysis ---

JSN Targets by Down and Distance:
                      targets  catch_pct  avg_air_yards
down distance_bucket                                   
1.0  0-1                    0        NaN            NaN
     2-3                    0        NaN            NaN
     4-6                    2     100.00           5.50
     7-10                  46      80.43          13.13
     11+                    1     100.00          12.00
2.0  0-1                    2      50.00          29.00
     2-3                    3     100.00          12.00
     4-6                    3     100.00           7.00
     7-10                  22      77.27          12.64
     11+                   11      63.64           9.45
3.0  0-1                    1       0.00          12.00
     2-3                    2      50.00          -5.00
     4-6                   11      81.82          15.18
     7-10                   9      66.67           9.11
     11+                    7